In [1]:
import nltk, string, re, os, numpy, math
from nltk import *
from nltk.stem import WordNetLemmatizer
from bitarray import bitarray
import pickle
#nltk.download('punkt') # uncomment the following two lines(including this line) if it is your first time running this code 
#nltk.download('wordnet')

In [2]:
#this function will help us throuhg cleanign the texts so we can find the stop words. It has 
def clean_text(s):
    s = str.lower(s)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    s = regex.sub('', s)
    tokens = nltk.word_tokenize(s)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

In [3]:
sentence = input("Enter the sentence you want to be cleaned:")
sentence = ' '.join(clean_text(sentence))
print(sentence)

Enter the sentence you want to be cleaned:salam
salam


In [4]:
#building a dictionary
path = "../data/EnglishData"
dic = {}
for file in os.listdir(path):
    filename = os.path.join(path, file)
    with open(filename) as f:
        s = f.readlines()
        s = ' '.join(s) 
        tks = clean_text(s)
        for tk in tks:
            if tk in dic:
                dic[tk] = dic[tk]+1
            else:
                dic[tk] = 1

#find the stop words based on their frequency in the document
freq = sorted(dic.items(), key = lambda kv:-kv[1])
stop_words = [kv[0] for kv in freq[1:50]]

In [19]:
print(stop_words)

['a', 'of', 'to', 'and', 'is', 'in', 'i', 'that', 'it', 'for', 'be', 'this', 'not', 'you', 'are', 'have', 'with', 'on', 'or', 'but', 'wa', 'if', 'my', 'can', 'by', 'an', 'from', 'about', 'one', 'at', 'there', 'ha', 'they', 'what', 'your', 'do', 'some', 'any', 'would', 'will', 'who', 'me', 'all', 'no', 'which', 'know', 'so', 'more', 'been']


In [31]:
def normalize(s):
    s = str.lower(s)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    s = regex.sub('', s)
    tokens = nltk.word_tokenize(s)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [37]:
sentence = input("Enter the sentence you want to be cleaned:")
sentence = (normalize(sentence))
print(sentence)

Enter the sentence you want to be cleaned:limits of AZT\'s efficacy and now suggest using drug  either sequentiallywith other drugs or kind of AIDS  treatment "cocktail" combining a numberof drugs to fight the  virus all at once.  "Treating people with AZT alonedoesn\'t  happen in real world anymore, said Dr. Mark Jacobson of the University of California--San Francisco.  Also, with recent  findingsindicating that HIV replicates rapidly in the lymph  nodes after infection,physicians may begin pushing even harder  early treatment of HIV-infectedpatients==================================================================
['limit', 'azt', 'efficacy', 'now', 'suggest', 'using', 'drug', 'either', 'sequentiallywith', 'other', 'drug', 'kind', 'aid', 'treatment', 'cocktail', 'combining', 'numberof', 'drug', 'fight', 'the', 'virus', 'once', 'treating', 'people', 'azt', 'alonedoesnt', 'happen', 'real', 'world', 'anymore', 'said', 'dr', 'mark', 'jacobson', 'the', 'university', 'californiasan', 'fran

In [78]:
#constructing the posting list 
path = "../data/EnglishData"
pl = {}
docs = []
for file in os.listdir(path):
    docs.append(file)
    filename = os.path.join(path, file)
    with open(filename) as f:
        lines = f.readlines()
        line = ' '.join(lines)
        tks = normalize(line)
        for n, tk in enumerate(tks):
            tmp = (file, n)
            if tk in pl:
                pl[tk].append(tmp)
            else:
                pl[tk] = [tmp]
                
terms = [term for term,_ in pl.items()]
term_index = {}
for i,term in enumerate(terms):
    term_index[term] = ifor file in os.listdir(path):
    docs.append(file)
    filename = os.path.join(path, file)
    with open(filename) as f:
        lines = f.readlines()
        line = ' '.join(lines)
        tks = normalize(line)
        for n, tk in enumerate(tks):
            tmp = (file, n)
            if tk in pl:
                pl[tk].append(tmp)
            else:
                pl[tk] = [tmp]
                

SyntaxError: invalid syntax (<ipython-input-78-e4c6398c2c20>, line 22)

In [254]:
###save the posting list to file
#convert the posting list to a posting list considting of diffs.
cpl={}
for term, ps in pl.items():
    #documents
    cdocs = [int(dp[0]) for dp in ps]
    tail = [0]+[doc for doc in cdocs[:-1]]
    cdocs = [next_d - d for d,next_d in zip(tail, cdocs)]
    sign = ['1' if x<0 else '0' for x in cdocs]
    cdbin = [("{0:b}".format(abs(x)))[1:] for x in cdocs]
    cdbin = [''.join(['1' for _ in range(len(bin))]+['0'])+bin for bin in cdbin]
    cdbin = [sign+bin for sign,bin in zip(sign,cdbin)]

    #positions
    cpos = [dp[1] for dp in ps]
    tail = [0]+[pos for pos in cpos[:-1]]
    cpos = [next_p - p for p,next_p in zip(tail, cpos)]
    sign = ['1' if x<0 else '0' for x in cpos]
    cpbin = [("{0:b}".format(abs(x)))[1:] for x in cpos]
    cpbin = [''.join(['1' for _ in range(len(bin))]+['0'])+bin for bin in cpbin]
    cpbin = [sign+bin for sign,bin in zip(sign,cpbin)]
    
    #joining both of these
    cbin = ''.join([x+y for x,y in zip(cdbin, cpbin)])
    cpl[term] = bitarray(cbin)
f = open("cpl.bin", "wb")
pickle.dump(cpl, f)
f.close()

[('59165', 727),
 ('59166', 871),
 ('59167', 887),
 ('59168', 930),
 ('59169', 989),
 ('59249', 212),
 ('59250', 237),
 ('59251', 243),
 ('59129', 1576),
 ('58814', 1),
 ('58815', 79),
 ('59481', 100),
 ('59482', 107),
 ('59294', 515),
 ('59295', 527),
 ('59296', 558),
 ('59297', 655),
 ('59298', 1201),
 ('59299', 1445),
 ('59300', 1531),
 ('59301', 2735),
 ('59338', 184),
 ('59339', 197),
 ('59340', 269),
 ('59341', 276),
 ('59342', 701),
 ('59343', 796),
 ('59344', 808),
 ('59345', 1064),
 ('59373', 0),
 ('59395', 67),
 ('59147', 2489),
 ('59148', 2516),
 ('59149', 2525),
 ('59150', 2562),
 ('59151', 2570),
 ('58894', 87),
 ('59410', 38),
 ('59043', 174),
 ('58832', 10),
 ('58947', 93),
 ('59330', 195),
 ('59390', 4),
 ('59348', 337),
 ('59349', 360),
 ('59149', 256),
 ('59282', 441),
 ('58164', 39),
 ('59668', 18),
 ('59669', 25),
 ('58969', 308),
 ('59211', 1524),
 ('59311', 352),
 ('59312', 431),
 ('59313', 470),
 ('59314', 584),
 ('59315', 860),
 ('59317', 390),
 ('59318', 904),


In [51]:
f = open("cpl.bin", "rb")
cpl = pickle.load(f)
f.close()
rpl={}
for term, ps in cpl.items():
    number = []
    ps = ps.to01()
    pos = 0
    while(pos<len(ps)):
        sign = int(ps[pos])
        length = 0
        pos += 1
        while pos < len(ps) and ps[pos] == '1':
            pos+=1
            length+=1
        num = int(ps[pos:pos+length+1], 2)+(1<<length)
        if sign == 1:
            num *= -1
        number.append(num)
        pos = pos+length+1
    dcs,pos=[0],[0]
    for i,x in enumerate(number):
        if(i%2 == 0):
            dcs.append(x+dcs[-1])
        else:
            pos.append(x+pos[-1])
    dcs = dcs[1:]
    pos = pos[1:]
    if(len(dcs)!= len(pos)):
        print("not true!")
    dp = [(str(dcs[i]),pos[i])for i in range(len(dcs))]
    rpl[term] = dp
rpl = pl

In [77]:
rpl["person"]

[('59315', 4),
 ('59069', 217),
 ('59069', 502),
 ('59615', 59),
 ('58855', 24),
 ('59034', 244),
 ('59245', 45),
 ('59073', 40),
 ('59073', 86),
 ('59147', 34),
 ('59129', 138),
 ('59576', 125),
 ('58951', 30),
 ('59202', 2),
 ('59499', 573),
 ('58774', 25),
 ('59441', 17),
 ('58058', 44),
 ('58112', 37),
 ('58112', 38),
 ('59469', 3),
 ('59528', 95),
 ('59592', 35),
 ('59286', 166),
 ('59286', 176),
 ('59286', 2104),
 ('59286', 2219),
 ('59502', 6),
 ('58946', 206),
 ('59136', 4),
 ('59136', 115),
 ('59060', 87),
 ('59067', 3),
 ('58905', 606),
 ('58997', 6),
 ('58986', 80),
 ('59293', 1),
 ('58152', 429),
 ('58152', 933),
 ('58152', 955),
 ('59255', 415),
 ('59255', 425),
 ('59255', 426),
 ('58137', 10),
 ('59076', 80),
 ('58099', 62),
 ('58909', 11),
 ('58909', 156),
 ('59496', 17),
 ('58906', 136),
 ('58941', 103),
 ('58941', 376),
 ('59183', 214),
 ('59183', 839),
 ('59283', 1357),
 ('59283', 1365),
 ('59283', 1395),
 ('59283', 1424),
 ('59283', 1443),
 ('59283', 1471),
 ('59283'

In [26]:
bigram = {}
for token in [kv[0] for kv in pl.items()]:
    for n, ch in enumerate(token[:-1]):
        bi = token[n]+token[n+1]
        if(bi in bigram):
            bigram[bi].append(token)
        else:
            bigram[bi]=[token]

In [28]:
#correcting the word in the queries
sl = 0
wl = {}
def correct_word(w):
    for n, ch in enumerate(w[:-1]):
        bi = w[n]+w[n+1]
        o
        for word in bigram[bi]:
            if word in wl:
                wl[word] = wl[word] + 1
            else:
                wl[word] = 1
    candids = []
    for word, n in wl.items():
        if n > sl:
            candids.append(word)
    distances = [edit_distance(candid,w) for candid in candids]
    candids = [candid for _,candid in sorted(zip(distances,candids))]
    print(candids[0:5])

In [42]:
query = input("Enter the query you want to be corrected:")
for token in nltk.word_tokenize(query):
    correct_word(token)

Enter the query you want to be corrected:smakqr


KeyError: 'qr'

In [13]:
idf={}
for term,posting_list in pl.items(): #calculate idf, idf is a sign of how important a term is
    idf[term] = math.log(len(docs)/(len(posting_list)))
tfidf={} #dictionary from doc to its corresponding vector
for doc in docs:
    tfidf[doc] = [0 for term in terms]
    
for term, posting_list in pl.items():
    for doc_pos in posting_list:
        doc = doc_pos[0]
        tfidf[doc][term_index[term]] += 1

for doc in docs:
    tfidf[doc] = [ 1+math.log(tf) if tf > 0 else 0 for tf in tfidf[doc]]
    doc_len = math.sqrt(sum([x*x for x in tfidf[doc]]))
    if doc_len != 0:
        tfidf[doc] = [x/doc_len for x in tfidf[doc]]

In [69]:
def add_doc(file):
    docs.append(file)
    filename = os.path.join(path, file)
    with open(filename) as f:
        lines = f.readlines()
        line = ' '.join(lines)
        tks = normalize(line)
        for n, tk in enumerate(tks):
            tmp = (file, n)
            if tk in rpl:
                rpl[tk].append(tmp)
            else:
                rpl[tk] = [tmp]
def del_doc(doc):
    for x,y in rpl.items():
        yp = []
        for f,s in y:
            if f != doc:
                yp.append((f,s))
        rpl[x] = yp

In [85]:
del_doc("59229")

In [83]:
add_doc("59229")

In [86]:
rpl["surgery"]

[('59550', 70),
 ('58813', 72),
 ('58813', 95),
 ('59245', 121),
 ('59123', 449),
 ('59456', 24),
 ('59456', 60),
 ('59456', 96),
 ('59456', 144),
 ('59511', 27),
 ('59511', 31),
 ('59557', 113),
 ('58112', 259),
 ('58578', 1322),
 ('59286', 1144),
 ('59286', 1171),
 ('59286', 1183),
 ('59286', 1968),
 ('59286', 2077),
 ('58090', 14),
 ('58090', 20),
 ('58090', 49),
 ('58090', 82),
 ('59424', 32),
 ('59424', 40),
 ('59424', 59),
 ('59276', 65),
 ('58135', 72),
 ('59475', 24),
 ('59475', 101),
 ('58986', 31),
 ('59255', 395),
 ('59255', 409),
 ('59199', 158),
 ('59453', 66),
 ('58829', 134),
 ('58829', 157),
 ('59343', 32),
 ('59343', 40),
 ('59343', 59),
 ('58095', 7),
 ('58820', 4),
 ('58820', 73),
 ('58820', 114),
 ('58820', 132),
 ('58975', 14),
 ('58876', 67),
 ('59244', 32),
 ('58881', 117),
 ('58109', 53),
 ('58109', 112),
 ('58109', 121),
 ('58109', 628),
 ('58109', 639),
 ('59548', 604),
 ('59451', 24),
 ('59432', 92),
 ('59432', 96),
 ('59432', 146),
 ('59072', 92),
 ('59448',

In [14]:
def query_2_vec(query):
    tks = normalize(query)
    vec = [0 for term in terms]
    for term in tks:
        vec[term_index[term]]+= 1
    vec = [ 1+math.log(tf) if tf > 0 else 0 for tf in vec]
    for term in tks:
        vec[term_index[term]] = vec[term_index[term]]*idf[term]
    query_len = math.sqrt(sum([x*x for x in vec]))
    if query_len != 0:
        vec = [x/query_len for x in vec]
    return vec

def cosine(v, u):
    ans = sum([v[i] * u[i] for i in range(len(v))])
    return ans
    
def search_vec(vec):
    global tfidf, docs
    cosines = [-1 * cosine(vec, tfidf[doc]) for doc in docs]
    ans = [doc for _,doc in sorted(zip(cosines, docs))]
    return ans


def show_doc(file):
    filename = os.path.join(path, file)
    with open(filename) as f:
        lines = f.readlines()
        line = ' '.join(lines)
        print(line)

In [57]:
del_doc("59125")

In [46]:
query = input("Enter the query you want to search for:")
vec = query_2_vec(query)
ans = search_vec(vec)
show_doc(ans[0])

Enter the query you want to search for:AdministratorRobert HIV/AIDS
------------- cut here ----------------limits of AZT's efficacy and now suggest using the drug  either sequentiallywith other drugs or in a kind of AIDS  treatment "cocktail" combining a numberof drugs to fight the  virus all at once.  "Treating people with AZT alonedoesn\'t  happen in the real world anymore," said Dr. Mark Jacobson of the University of California--San Francisco.  Also, with recent  findingsindicating that HIV replicates rapidly in the lymph  nodes after infection,physicians may begin pushing even harder  for early treatment of HIV-infectedpatients==================================================================   "New Infectious Disease Push" American Medical News (04/05/93) Vol. 36, No.13, P. 2     The Center for Disease Control will launch a worldwide network to trackthe spread of infectious diseases and detect drug-resistant or new strains intime to help prevent their spread.  The network is expec